In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.8.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential=DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential=InteractiveBrowserCredential()

In [4]:
ml_client=MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [5]:
MLClient.from_config(credential=credential)

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f6979416110>,
         subscription_id=18a1f27f-edf5-495e-9acb-753c93335294,
         resource_group_name=rg-dp100-l73933f31d0c94dca80,
         workspace_name=mlw-dp100-l73933f31d0c94dca80)

In [6]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
# check for this uri in Named asset URI under the data asset name: diabetes-training
my_training_data_input=Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

In [7]:
from azure.ai.ml import automl

# configure the classification job
classification_job=automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# Set the Limits (optional)
classification_job.set_limits(
    enable_early_termination=True,
    timeout_minutes=60,
    trial_timeout_minutes=20,
    max_trials=5
    )

# Set the Training Properties (Optional)
classification_job.set_training(
    blocked_training_algorithms=['LogisticRegression'],
    # Stack Ensemble is disabled if set to True
    enable_onnx_compatible_models=True

)


In [8]:
returned_job=ml_client.jobs.create_or_update(job=classification_job)

aml_url=returned_job.studio_url
print(f"Monitor Results at: {aml_url}")

Monitor Results at: https://ml.azure.com/runs/dynamic_berry_z5ydkr8dnf?wsid=/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourcegroups/rg-dp100-l73933f31d0c94dca80/workspaces/mlw-dp100-l73933f31d0c94dca80&tid=6a1d2f96-8cdf-4d1a-943d-7b73f4dfbb6d
